# Plots - BS, DOS, pDOS, QE, MTE, Murnaghan EOS, Convergence

In [ ]:
import subprocess,os,mmap,sys
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

from ase.io import read
from ase.visualize import view
from ase.build import surface
import ase.calculators.castep
import ase.io.castep

from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.electronic_structure.bandstructure import BandStructureSymmLine
from pymatgen.symmetry.kpath import KPathSetyawanCurtarolo,KPathLatimerMunro
from pymatgen.electronic_structure.dos import *
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


sys.path.append("C:/Users/fcm19/Desktop/PhD/Project/modules_codes/script_share")
from python.bs_plotter_felix_mod.src import *
from python_functions import *
from castep_output_class import *
from optados_output_class import *



In [ ]:
def read_in_debug_file(filename):
    mat_shape = []
    with open(filename,'r') as f:
        next(f)
        line= next(f)
        mat_shape = [int(x) for x in line.split()]
        print("Will return matrix of shape:", mat_shape)
    matrix = np.genfromtxt(filename,skip_header=2,delimiter=' ')
    matrix = np.reshape(matrix,mat_shape, order='F')
    return matrix;
    
def read_qe_calculation_values(filename:str, model:str):
    data = {}
    with open(filename, "r+b") as input:
        mm = mmap.mmap(input.fileno(),0)
        iterator = iter(mm.readline, b"")
        for line in iterator:
            line_temp = line.decode("ANSI").strip()
            if "- Printing list of values going into" in line_temp:
                data['printed_quantities'] = next(iterator).decode("ANSI").strip().split(' - ')
                if model == '3step':
                    data['e_fermi'] = float(next(iterator).decode("ANSI").strip().split()[2])
                    data['data_shape'] = [int(x) for x in next(iterator).decode("ANSI").strip().split()[2:-1]]
                    # print(data['data_shape'])
                    # data['data_shape'] = data['data_shape'][1:] + [data['data_shape'][0]]
                    # print(data['data_shape'])
                if model == '1step':
                    data['data_shape'] = [int(x) for x in next(iterator).decode("ANSI").strip().split()[2:]]
                    data['data_shape'] = data['data_shape'][1:] + [data['data_shape'][0]]
                    # print(data['data_shape'])
                data['values'],data['indices'],data['qe_values'] = [],[],[]
                line_temp = next(iterator).decode("ANSI").strip()
                while "- Finished Printing -" not in line_temp:
                    data['indices'].append([int(x) for x in line_temp.split()[1:-1]])
                    line_temp = next(iterator).decode("ANSI").strip()
                    data['qe_values'].append(float(line_temp.split()[0]))
                    data['values'].append([float(x) for x in line_temp.split()[1:]])
                    line_temp = next(iterator).decode("ANSI").strip()
        mm.close()

        data['values'] = np.array(data['values'])
        data['indices'] = np.array(data['indices'])
        data['qe_values'] = np.array(data['qe_values'])
        #print(data['values'].shape)
        #print(data['values'])
        #data['values'] = np.reshape(data['values'],data['data_shape'])
    return data;
    
# def filter_3step_values(data_in:dict):
#     '''This function takes in a dictionary of raw data and filters out the bands, where the final state is below the\n
#     fermi level and thus assumed to be occupied. This selection rule is a WIP. The information for the atom,\n
#     final state, initial state, spin, and k point are preserved and returned as dict['indices'].'''
#     data_out = {}
#     data_out['printed_quantities'] = data_in['printed_quantities']
#     data_out['original_shape'] = data_in['data_shape']
#     indices = np.empty(data_in['data_shape'][:-1],dtype=object)
#     for m in range( data_in['data_shape'][4]): # K point
#         for l in range( data_in['data_shape'][3]): # Spin
#             for k in range( data_in['data_shape'][2]): # Final State
#                 for j in range( data_in['data_shape'][1]): # Initial State
#                     for i in range( data_in['data_shape'][0]): # Layer
#                         indices[i,j,k,l,m] = f"{i}-{j}-{k}-{l}-{m}"
#     indices = indices.flatten(order='F')
#     filter_array = data_in['values'][:,0] > data_in['e_fermi']
#     data_out['values'] = data_in['values'][filter_array]
#     data_out['indices'] = indices[filter_array]
#     return data_out;
    

In [ ]:
directory = "./OD_debug_files/OptaDOS_debug_output/4-added_singular_value_printing/"
file = 'Ag100_fine_grid_4.7_1step_0.1_iprint_5.odo'
filename = directory + file 
one_step_point1 = read_qe_calculation_values(filename,model='1step')
print(one_step_point1['printed_quantities'])
print(one_step_point1['data_shape'])
print(one_step_point1['values'][0,0,0,0,:])

In [ ]:
for i in range(len(one_step_point1['values'][0,:,0,0,0])):
    if one_step_point1['values'][0,i,0,0,1] > 0:
        print(i)
        print(one_step_point1['values'][0,i,0,0,:])

In [ ]:
directory = "./OD_debug_files/OptaDOS_debug_output/4-added_singular_value_printing/"
file = 'Ag100_fine_grid_4.7_1step_0.001_iprint_5.odo'
filename = directory + file 
one_step_point001 = read_qe_calculation_values(filename,model='1step')
print(one_step_point001['printed_quantities'])
print(one_step_point001['data_shape'])
print(one_step_point001['values'][0,0,0,0,:])


In [ ]:
directory = "./OD_debug_files/OptaDOS_debug_output/4-added_singular_value_printing/"
file = 'Ag100_fine_grid_4.7_3step_0.1_iprint_5.odo'
filename = directory + file 
three_step_point1 = read_qe_calculation_values(filename,model='3step')

In [ ]:
filtered_three_step_point1 = filter_3step_values(three_step_point1)

In [ ]:
temp_array = filtered_three_step_point1['values'][:,2:13]
print(temp_array.shape)
print(temp_array[0])
print(len(temp_array))
for i in range(len(temp_array)):
    if not np.any(temp_array[i] == 0):
        print(i)
        print(filtered_three_step_point1['indices'][i])


In [ ]:
for i in range(len(temp_array[0])):
    print(filtered_three_step_point1['printed_quantities'][i+1],np.sum(temp_array[:,i]))

In [ ]:
print(three_step_point1['values'][0:3,0,0,0,0,:])

In [ ]:
directory = './OD_debug_files/OD_visualisation_tests/XGMX_bs_path/bs_test/'
file = 'Ag100_fine_grid_4.2_3step_0.1_20_5_bs_path.odo'
filename = directory + file 
three_step_bspath = read_qe_calculation_values(filename,model='3step')
print(three_step_bspath['values'].shape)
print(three_step_bspath['printed_quantities'])
print(three_step_bspath['qe_values'][0])
print(three_step_bspath['values'][0,:])
three_step_bspath['indices'][0,:]
# filtered_three_step_bspath = filter_3step_values(three_step_bspath)
# print(filtered_three_step_bspath['values'][0])
# print(filtered_three_step_bspath['indices'][0])
# print(three_step_bspath['data_shape'])

In [ ]:
filter_array = three_step_bspath['qe_values']> 0.0 
filtered_qe =  three_step_bspath['qe_values'][filter_array]
filtered_indices = three_step_bspath['indices'][filter_array]
filtered_values = three_step_bspath['values'][filter_array]
print(filtered_indices.shape)
print(filtered_qe.shape)
print(filtered_values.shape)
print(filtered_indices[0])
print(filtered_qe[0])
print(filtered_values[0])
with open("./OD_debug_files/OD_visualisation_tests/XGMX_bs_path/bs_test/filtered_qe_contributions.dat",'w') as f:
    f.write(str(three_step_bspath['printed_quantities'][1:])+'\n')
    f.write('Fermi Energy = ' + str(round(three_step_bspath['e_fermi'],6)) + '\n')
    for index in range(len(filtered_indices)):
        f.write('Atom, Psi_i, Psi_f, Spin, K-pt\n')
        f.write(str(filtered_indices[index])+'\n')
        f.write(str(filtered_qe[index])+'\n')
        for i in range(filtered_values.shape[1]):
            f.write(str(filtered_values[index,i])+'  ')
        f.write('\n')
print("I am DONE :D")

In [ ]:
directory = "./OD_debug_files/OptaDOS_debug_output/4-added_singular_value_printing/"
file = 'Ag100_fine_grid_4.7_3step_0.001_iprint_5.odo'
filename = directory + file 
three_step_point001 = read_qe_calculation_values(filename,model='3step')
print(three_step_point001['values'].shape)
three_step_point001['loop_lengths'] = [13] + three_step_point001['loop_lengths']
three_step_point001['values'] = np.reshape(three_step_point001['values'].flatten(order='C'),three_step_point001['loop_lengths'],order='F')
print(three_step_point001['values'].shape)
print(three_step_point001['printed_data'])

In [ ]:
directory ="./OD_debug_files/OptaDOS_debug_output/3-added_delta_function_printing/" 
file = "Ag100_3step_delta_function.txt"
filename = directory + file
delta_function = read_in_debug_file(filename)

In [ ]:
from matplotlib.colors import LogNorm
fields = np.zeros([237,237])
print(fields.shape)
indices = []
for k in range(36):
    if np.sum(delta_function[:,:,k]) != 0:
        print("Index", k, "=", np.sum(delta_function[:,:,k]))
        for j in range(237):
            for i in range(237):
                if delta_function[i,j,k] != 0:
                    #print(k,j, np.sum(delta_function[:,j,k]))
                    fields[i,j] += 1
        indices.append(k)
fig, axes = plt.subplots(3,3,figsize=(20,20),dpi=200)
for i,ax in enumerate(axes.ravel()):
    if i == len(axes.ravel())-1:
        ax.imshow(fields)#,norm=LogNorm(vmin=0,vmax=np.max(fields)))
        ax.set(title="Boolean plot showing if the \ndelta function is /= 0 on any K-Point",xlabel='Final state',ylabel='Initial State')
        ax.xaxis.set_label_position('top')
        ax.xaxis.tick_top()
    else:
        minimum = np.min(delta_function[:,:,indices[i]])
        maximum = np.max(delta_function[:,:,indices[i]]) 
        ax.imshow(delta_function[:,:,indices[i]])#,norm=LogNorm(vmin=minimum,vmax=maximum))
        ax.set(title=f'K-Point #{1 + indices[i]}',xlabel='Final state',ylabel='Initial State')
        ax.xaxis.set_label_position('top')
        ax.xaxis.tick_top()
plt.savefig('./OD_debug_files/Ag100_delta_func_237_237_36.png')
#axes.matshow(fields)
#(xlabel='Final State',ylabel = 'Initial State')
print("max(fields) = " ,np.max(fields))
print("min(delta function values) = ",np.min(delta_function))

In [ ]:
from matplotlib.colors import LogNorm
fields = np.zeros([237,237])
for k in range(36):
    if np.sum(delta_function[:,:,k]) != 0:
        # print("Index", k, "=", np.sum(delta_function[:,:,k]))
        # for j in range(237):
        #     if np.sum(delta_function[:,j,k]):
        #         # print(k,j, np.sum(delta_function[:,j,k]))
        #         fields[j,k] += 1
        #         #fig, axes = plt.subplots(1,1,figsize=(20,20))
        minimum = np.min(delta_function[:,:,k])
        maximum = np.max(delta_function[:,:,k]) 
        fig, ax = plt.subplots(1,1,figsize=(10,10),dpi=200)
        ax.imshow(delta_function[:,:,k])#,norm=LogNorm(vmin=minimum,vmax=maximum))
        ax.set(xlabel='Final state',ylabel='Initial State',)
        ax.xaxis.set_label_position('top')
        ax.xaxis.tick_top()

In [ ]:
directory ="./OD_debug_files/OptaDOS_debug_output/3-added_delta_function_printing/" 
file = "Ag100_3step_QE_matrix.txt"
filename = directory + file
qe_matrix = read_in_debug_file(filename)

In [ ]:
def real_to_rec_lattice(real):
    reciprocal = np.zeros((3,3))
    V = np.dot(real[0],np.cross(real[1],real[2]))
    reciprocal[0] = np.cross(real[1],real[2])*(2*np.pi/V)
    reciprocal[1] = np.cross(real[2],real[0])*(2*np.pi/V)
    reciprocal[2] = np.cross(real[0],real[1])*(2*np.pi/V)
    return reciprocal;

def get_scaled_distances(cartesian):
    moved = np.insert(cartesian, 0, cartesian[0]).reshape((np.shape(cartesian)[0]+1,np.shape(cartesian)[1]))
    cart_diff = cartesian - moved[:-1]
    distances = np.linalg.norm(cart_diff,axis=1)
    for idx, i in enumerate(distances):
        if idx == 0:
            continue
        distances[idx] = distances[idx-1] + distances[idx]
    scaled_distances = distances/np.nanmax(distances)
    return scaled_distances;

def read_bands_file(path:str):
    hartree2eV = 27.21139664
    bohr2ang = 0.529177249
    lattice = []
    data = {}
    with open(path,'r') as f:
        lines = f.readlines()
    num_kpoints = int(lines[0].strip().split()[3])
    num_spins = int(lines[1].strip().split()[4])
    num_bands = int(lines[3].strip().split()[3])
    fermi_e = float(lines[4].strip().split()[5])*hartree2eV
    for i in range(3):
        lattice.append([float(x)*bohr2ang for x in lines[6+i].strip().split()])
    rec_lattice = real_to_rec_lattice(lattice)
    data['reciprocal_lattice'] = rec_lattice
    kpt_cart = np.zeros((num_kpoints,3))
    kpt_frac = np.zeros((num_kpoints,3))
    eigenvalues_efermi = np.zeros((num_bands, num_spins, num_kpoints))
    length_kpt_block = (num_bands+1)*num_spins+1
    for k in range(num_kpoints):
        kpt_line_index = 9+k*length_kpt_block
        frac=np.array([float(x) for x in lines[kpt_line_index].strip().split()[2:5]])
        kpt_cart[k] = np.dot(rec_lattice,frac)
        kpt_frac[k] = frac
        for j in range(num_spins):
            for i in range(num_bands):
                line_index = kpt_line_index + j*(num_bands+1)+i+2
                eigenvalues_efermi[i,j,k] = float(lines[line_index].strip().split()[0])*hartree2eV-fermi_e
    data['scaled_kpt_path'] = get_scaled_distances(kpt_cart)
    data['eigenval_efermi_0'] = eigenvalues_efermi
    data['kpt_cart'] = kpt_cart
    data['kpt_frac'] = kpt_frac
    data['num_eigen'] = num_bands
    data['num_kpt'] = num_kpoints
    return data;

In [ ]:
data_matrices = {'norm':{},'test':{}}
file_bands = './OD_debug_files/OD_visualisation_tests/Ag100_fine_grid.bands'
data_bands = read_bands_file(file_bands)

In [ ]:
energies = np.linspace(4.79,4.85,7)
keywords = [['norm',''],['test','_test']]
#energies =[4.85]
for keyword in keywords:
    for energy in energies:
        file = f'./OD_debug_files/OD_visualisation_tests/iprint_6/Ag100_fine_grid_{energy:.2f}_3step_0.01_20_6{keyword[1]}.odo'
        data = read_qe_matrix_values(file)
        data_matrices[keyword[0]][round(energy,2)] = data
        print(f'{energy:.2f}')

In [ ]:
import matplotlib.colors as mlp
plt.style.use('seaborn-darkgrid')
fig = plt.figure(figsize=(12,5),dpi=200)
ax = fig.add_subplot()
qe_sum = np.zeros((7,237,36))
energies = np.linspace(4.79,4.85,7)

for idx,energy in enumerate(energies[:-1]):
    qe_sum[idx,:,:] = np.sum(data_matrices['test'][round(energy,2)]['matrix'],axis=(1,3,4))
norm = mlp.Normalize(vmin=0, vmax=np.nanmax(qe_sum))
for idx,energy in enumerate(energies[:-1]):
    x_val = np.full((237),round(energy,2))
    for i in range(np.shape(qe_sum)[2]):
        if np.sum(qe_sum[idx,:,i]) > 0:
            alphas = qe_sum[idx,:,i]/np.nanmax(qe_sum[idx,:,i])
            ax.scatter(x = x_val, y = data_bands['eigenval_efermi_0'][:,0,i],cmap='cividis',c= norm(qe_sum[idx,:,i]),marker='.',alpha=alphas)
            ax.set_xlabel('Photon Energy [eV]')
            ax.set_ylabel(r'$E - E_F$ [eV]')
            ax.set_facecolor('white')
    for j in range(np.shape(qe_sum)[2]):
        if np.sum(qe_sum[idx,:,j]) > 0:
            print(np.nanmax(qe_sum[idx,:,j]))
            print(np.sum(qe_sum[idx,:,:]))
            for k in range(237):
                if abs(np.nanmax(qe_sum[idx,:,j]) - qe_sum[idx,k,j]) < 1E-12:
                    ax.annotate(str(qe_sum[idx,k,j]),xy=[energies[idx],data_bands['eigenval_efermi_0'][k,0,j]],xytext=[5,5],textcoords='offset pixels')
plt.title(f"QE contribution from bands at Kpoint {data_matrices['norm'][4.79]['kpoints'][16]}")
plt.ylim(-10,35)
plt.savefig('./OD_debug_files/OD_visualisation_tests/Ag100_band_emission_near_edge_comp_OD_test.png')

In [ ]:

energies = np.linspace(4.4,4.7,4)
model = '3step'
iprint = 7
for energy in energies:
    file = f'./OD_debug_files/OD_visualisation_tests/15_15_iprint_7/Ag100_fine_grid_{energy:.2f}_3step_0.1_20_7.odo'
    print(file)
    data = read_qe_matrix_values(file)
    #data_matrices['test'][round(energy,2)] = data
    #print(round(energy,2))
    if data['model'] == '3step' and iprint == 6:
        qe_sum = np.sum(data['matrix'],axis = (0,1,3,4))
    elif data['model'] == '1step' or iprint == 7:
        qe_sum = np.sum(data['matrix'],axis = (0,2,3))
    #plt.style.use('seaborn-bright')
    #print(plt.style.available)
    half_bz = 2.193102/2
    fig = plt.figure(figsize=(8,8),dpi=200)
    ax = fig.add_subplot()
    ax.scatter(data['kpoints'][:,0],data['kpoints'][:,1],cmap='viridis',c=qe_sum/np.nanmax(qe_sum))
    ax.plot([0,0,half_bz,0],[half_bz,0,half_bz,half_bz],c='tab:green')
    ax.xaxis.tick_top()
    ax.set_xlabel(r'Recip lattice vector $\vec{b}_1$ $\AA^{-1}$')  
    ax.set_ylabel(r'Recip lattice vector $\vec{b}_2$ $\AA^{-1}$')  
    ax.xaxis.set_label_position('top') 
    ax.set_facecolor('white')
    plt.title(f'Summed QE for each K-Point with Photon Energy @ {energy:.2f}\nTotal QE = {np.sum(qe_sum):.4E}')
    plt.annotate('X',[0,half_bz],[-20,20],textcoords='offset pixels',fontweight='bold',fontsize = 'large')
    plt.annotate(r'$\bf{\Gamma}$',[0,0],[-24,-24],textcoords='offset pixels',fontweight='bold',fontsize = 'large')
    plt.annotate('M',[half_bz,half_bz],[15,15],textcoords='offset pixels',fontweight='bold',fontsize = 'large')
    for i in range(np.shape(qe_sum)[0]):
        if qe_sum[i] > 1E-50:
            plt.annotate(f'{qe_sum[i]:.6E}',data['kpoints'][i,:2],[10,10],textcoords='offset pixels')
    plt.tight_layout()
    plt.rcParams.update({
        "savefig.facecolor": (1.0, 1.0, 1.0, 1),
    })
    plt.savefig(f'./OD_debug_files/OD_visualisation_tests/Ag100_QE_15_15_kpt_visual_{energy:.2f}.png')

In [ ]:
path = './OD_debug_files/OD_visualisation_tests/Ag100_fine_grid.bands.spec'
more_bands = read_bands_file(path)
for item in more_bands['kpt_frac']:
    print(item[0], item[1], item[2], f"1/{np.shape(more_bands['kpt_frac'])[0]}")

## Plotting the QE for a Bandstructure Plot

In [ ]:
def read_qe_matrix_values(filename:str):
    data = {}
    with open(filename, "r+b") as input:
            mm = mmap.mmap(input.fileno(),0)
            iterator = iter(mm.readline, b"")
            for line in iterator:
                line_temp = line.decode("ANSI").strip()
                if 'K-Points in Cartesian Coordinates' in line_temp:
                    line = next(iterator).decode("ANSI")
                    kpoints = []
                    while not 'Finished Printing' in line:
                        #print(line)
                        kpoints.append([float(x) for x in line.strip().split()])
                        line = next(iterator).decode("ANSI")
                    data['kpoints'] = np.array(kpoints)
                if 'QE Matrix --' in line_temp:
                    data['model'] = line_temp.strip().split()[2]
                    temp = next(iterator)
                    shape = [int(x) for x in next(iterator).decode("ANSI").strip().split()]
                    matrix = []
                    line = next(iterator).decode("ANSI")
                    while not 'Finished Printing' in line:
                        matrix.append([float(x) for x in line.strip().split()]) 
                        line = next(iterator).decode("ANSI")
                    matrix = np.array(matrix)
            mm.close()
            data['matrix'] = np.reshape(matrix.flatten(order='C'),shape,order = 'F')
    return data;
    
data_matrices = {'norm':{},'test':{},'xgmx_bs':{}}
energies = np.linspace(4.2,6.5,24)
for energy in energies:
    file = f'./OD_debug_files/OD_visualisation_tests/XGMX_bs_path/Ag100_fine_grid_{energy:.2f}_3step_0.1_20_7_bs_path.odo'
    data = read_qe_matrix_values(file)
    data_matrices['xgmx_bs'][round(energy,2)] = data
file_bands = './OD_debug_files/OD_visualisation_tests/Ag100_fine_grid_bs_path-spectral.bands'
data_bands = read_bands_file(file_bands)

In [ ]:
np.shape(data_matrices['xgmx_bs'][4.2]['matrix'])

In [ ]:
import matplotlib.colors as mlp
import matplotlib as mpl
%matplotlib widget
from matplotlib.widgets import Slider, Button

# qe_sums(eigenvalues, kpoints, energies)
# band_energies(eigenvalues, kpoints)
# band_distances(eigenvalues, kpoints)
# norms(eigenvalues, kpoints, energies)
# alphas(eigenvalues, kpoints, energies)

# SETTING UP THE DATA STRUCTURES

num_eigen = data_bands['num_eigen']
num_kpts = data_bands['num_kpt']
num_energies = len(energies)
num_layers = np.shape(data_matrices['xgmx_bs'][energies[0]]['matrix'])[-1]
#print(num_layers)

qe_sums = np.zeros((num_eigen,num_kpts, num_energies))
qe_layers = np.zeros((num_layers,num_energies))
band_energies = np.zeros((num_eigen,num_kpts))
band_energies = np.sum(data_bands['eigenval_efermi_0'],axis=1)
band_distances = np.zeros((num_eigen, num_kpts))
for i in range(num_eigen): band_distances[i,:] =  data_bands['scaled_kpt_path']
norms = np.zeros((num_eigen,num_kpts, num_energies))
norms_layers = np.zeros((num_layers,num_energies))
alphas = np.zeros((num_eigen,num_kpts, num_energies))

for idx,energy in enumerate(energies):
    temp_matrix = data_matrices['xgmx_bs'][round(energy,2)]['matrix']
    qe_sums[:,:,idx] = np.sum(temp_matrix,axis=(2,3))
    qe_layers[:,idx] = np.sum(temp_matrix,axis=(0,1,2))
    #print(np.shape(np.sum(temp_matrix,axis=(0,1,2))))
non_zero = qe_sums[np.nonzero(qe_sums)]
non_zero_layers = qe_layers[np.nonzero(qe_layers)]
norm = mlp.LogNorm(vmin=np.nanmin(non_zero), vmax=np.nanmax(qe_sums),clip = True)
norm_layers = mlp.LogNorm(vmin=np.nanmin(non_zero_layers),vmax=np.nanmax(qe_layers),clip=True)
for idx,energy in enumerate(energies):
    alphas[:,:,idx] = norm(qe_sums[:,:,idx])*200

import scicomap as sc
from scicomap.utils import _fn_with_roots# load the color map (built-in or any mpl compatible cmap)
sc_map = sc.ScicoSequential(cmap='plasma')
sc_map.unif_sym_cmap(lift=None, bitonic=False, diffuse=True)
fixed_cmap = sc_map.get_mpl_color_map()
#plt.ion()
i = 0
imax = len(energies)-1
annotations = {}
plt.style.use('seaborn-darkgrid')
fig,ax = plt.subplots(1,2,figsize=(12,5),dpi=200,gridspec_kw={'width_ratios':[6,1]})
#fig.set(width_ratios=[3,1])
scatter = ax[0].scatter(band_distances,band_energies,c='grey',marker='.',s=12,alpha = 0.5,zorder=2)
ticks = [data_bands['scaled_kpt_path'][0],data_bands['scaled_kpt_path'][11],data_bands['scaled_kpt_path'][27],data_bands['scaled_kpt_path'][-1]]
ax[0].vlines(ticks,colors = 'k',ymin = -50,ymax = 50,linewidth=0.6,zorder=1)
ax[0].set_xticks(ticks,labels=['X',r'$\Gamma$','M','X'])
ax[1].set_xticks([])
layer_ticks = np.arange(0,-num_layers,-1)
labels_layers = ['Surface'] + np.arange(-1,-num_layers+1,-1).tolist() + ['Bulk']
ax[1].set_yticks(layer_ticks,labels_layers)
#ax.hlines([0],colors='k',xmin=0,xmax=1,linewidth = 0.6, linestyles='--',zorder=2)
title_1 = ax[0].set_title(f'Energy = {round(energies[0],2)} eV\nTotal QE: {np.sum(qe_sums[:,:,0]):.3E}')
scatter2 = ax[0].scatter(band_distances,band_energies,cmap=fixed_cmap,norm=norm,c=qe_sums[:,:,0].flatten(),s=alphas[:,:,0],marker='.',zorder=3)
colorbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm,cmap= fixed_cmap),ax = ax[0],orientation='vertical',label = 'QE')
scatter_layer = ax[1].scatter(np.full(num_layers,1),np.arange(0,-1*num_layers,-1),norm=norm_layers,cmap = fixed_cmap,c=qe_layers[:,0],marker='o')
colorbar_layer = fig.colorbar(mpl.cm.ScalarMappable(norm=norm_layers,cmap= fixed_cmap),ax = ax[1],orientation='vertical',label = 'QE')
for j in range(num_layers):
    annotations[j] = ax[1].annotate(text=f"{qe_layers[j,0]:.4E}", xy=(1,-1*j),xytext=(-50,15), textcoords='offset pixels', xycoords='data')
ax[0].set_ylim(-8.2,10)
ax[0].set_xlabel('k-vector')
ax[0].set_ylabel(r'$E - E_F$ [eV]')
fig.subplots_adjust(left=0.15)

def update(val):
    global i
    scatter2.set_array(qe_sums[:,:,index_slider.val].flatten())
    scatter2.set_sizes(alphas[:,:,index_slider.val].flatten())
    title_1.set_text(f'Energy = {round(energies[index_slider.val],2)} eV\nTotal QE: {np.sum(qe_sums[:,:,index_slider.val]):.3E}')
    scatter_layer.set_array(qe_layers[:,index_slider.val].flatten())
    #print(norms_layers[:,index_slider.val])
    for j in range(num_layers):
        annotations[j].set_text(f"{qe_layers[j,index_slider.val]:.4E}")
    fig.canvas.draw_idle()

indexax= fig.add_axes((0.05,0.6,0.03,0.25))
index_slider = Slider(
    ax=indexax,
    label='index',
    valmin=0,
    valmax=imax,
    valstep=1,
    valinit=0,
    orientation="vertical"
    )

def arrow_key_control(event):
    global i
    i = index_slider.val
    if event.key == 'left':
        if i == 0:
            i = imax
        else:
            i -= 1
        index_slider.set_val(i)
        update(i)
    elif event.key == 'right':
        if i == imax:
            i = 0
        else:
            i += 1
        index_slider.set_val(i)
        update(i)
    else:
        pass
index_slider.on_changed(update)
id2 = fig.canvas.mpl_connect('key_release_event', arrow_key_control)

# for idx,energy in energies:



In [ ]:
import matplotlib.colors as mlp
import matplotlib as mpl

# qe_sums(eigenvalues, kpoints, energies)
# band_energies(scaled distance, energy)
# norms(eigenvalues, kpoints, energies)
# alphas(eigenvalues, kpoints, energies)

plt.ion()

plt.style.use('seaborn-whitegrid')

fig,axes = plt.subplots(1,1,figsize=(12,8),dpi=200)

temp_data = data_matrices['xgmx_bs'][4.2]['matrix']
temp_bands = data_bands['eigenval_efermi_0']

qe_sum = np.zeros((data_bands['num_eigen'],data_bands['num_kpt']))
qe_sum[:,:] = np.sum(temp_data,axis=(2,3))
print(np.nanmax(qe_sum))

norm = mlp.LogNorm(vmin=np.nanmin(qe_sum[np.nonzero(qe_sum)]), vmax=np.nanmax(qe_sum),clip = True)

x_values = np.zeros(np.shape(qe_sum))

for i in range(data_bands['num_kpt']):
    x_values[:,i] = np.full((data_bands['num_eigen']),data_bands['scaled_kpt_path'][i])
    scatter = axes.scatter(x_values[:,i], temp_bands[:,0,i] ,c='grey',marker='.',s=12,alpha = 0.05,zorder=2)

ticks = [data_bands['scaled_kpt_path'][0],data_bands['scaled_kpt_path'][11],data_bands['scaled_kpt_path'][27],data_bands['scaled_kpt_path'][-1]]
axes.vlines(ticks,colors = 'k',ymin = -50,ymax = 50,linewidth=0.6,zorder=1)
axes.set_xticks(ticks,labels=['X',r'$\Gamma$','M','X'])
axes.hlines([0],colors='k',xmin=0,xmax=1,linewidth = 0.6, linestyles='--',zorder=2)

for i in range(data_bands['num_kpt']):
    alphas = norm_2(qe_sum[:,i])
    axes.scatter(x_values[:,i], temp_bands[:,0,i] ,cmap='PuRd',c=norm(qe_sum[:,i]),s=alphas*200,marker='.',zorder=3)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm,cmap= mpl.cm.PuRd),ax = axes,orientation='vertical',label = 'QE')
plt.xlabel('k-vector')
plt.ylabel(r'$E - E_F$ [eV]')
plt.ylim(-10,35)
plt.xlim(0,1)


In [ ]:
print(qe_sum[:,18])